In [ ]:
import tensorflow.keras.layers
import tensorflow as tf 

In [ ]:
cell = layers.SimpleRNNCell(3) # 创建 RNN Cell，内存向量长度为 3
cell.build(input_shape=(None,4)) # 输出特征长度 n=4
cell.trainable_variables # 打印 wxh, whh, b 张量

可以看到，SimpleRNNCell 内部维护了 3 个张量，kernel 变量即𝑾 张量，recurrent_kernel
变量即𝑾 张量，bias 变量即偏置𝒃向量。但是 RNN 的 Memory 向量 并不由SimpleRNNCell 维护，
需要用户自行初始化向量 𝟎并记录每个时间戳上的 𝒕。

In [ ]:
# 初始化状态向量，用列表包裹，统一格式
h0 = [tf.zeros([4, 64])]
x = tf.random.normal([4, 80, 100]) # 生成输入张量，4 个 80 单词的句子
xt = x[:,0,:] # 所有句子的第 1 个单词
# 构建输入特征 n=100,序列长度 s=80,状态长度=64 的 Cell
cell = layers.SimpleRNNCell(64)
out, h1 = cell(xt, h0) # 前向计算
print(out.shape, h1[0].shape)

可以看到经过一个时间戳的计算后，输出和状态张量的 shape 都为[𝑏, ℎ]，

对于长度为𝑠的训练来说，需要循环通过Cell 类𝑠次才算完成一次网络层的前向运算

In [ ]:
h = h0 # h 保存每个时间戳上的状态向量列表
# 在序列长度的维度解开输入，得到 xt:[b,n]
for xt in tf.unstack(x, axis=1):
     out, h = cell(xt, h) # 前向计算,out 和 h 均被覆盖
# 最终输出可以聚合每个时间戳上的输出，也可以只取最后时间戳的输出
out = out

最后一个时间戳的输出变量 out 将作为网络的最终输出。实际上，也可以将每个时间戳上
的输出保存，然后求和或者均值，将其作为网络的最终输出。

# 多层 SimpleRNNCell 网络

In [ ]:
x = tf.random.normal([4,80,100])
xt = x[:,0,:] # 取第一个时间戳的输入 x0
# 构建 2 个 Cell,先 cell0,后 cell1，内存状态向量长度都为 64
cell0 = layers.SimpleRNNCell(64)
cell1 = layers.SimpleRNNCell(64)
h0 = [tf.zeros([4,64])] # cell0 的初始状态向量
h1 = [tf.zeros([4,64])] # cell1 的初始状态向量

在时间轴上面循环计算多次来实现整个网络的前向运算，每个时间戳上的输入 xt 首先通过
第一层，得到输出 out0，再通过第二层，得到输出 out1，代码如下：

In [ ]:
for xt in tf.unstack(x, axis=1):
     # xt 作为输入，输出为 out0
     out0, h0 = cell0(xt, h0)
     # 上一个 cell 的输出 out0 作为本 cell 的输入
     out1, h1 = cell1(out0, h1)

上述方式先完成一个时间戳上的输入在所有层上的传播，再循环计算完所有时间戳上的输入。
实际上，也可以先完成输入在第一层上所有时间戳的计算，并保存第一层在所有时间
戳上的输出列表，再计算第二层、第三层等的传播。代码如下

In [ ]:
# 保存上一层的所有时间戳上面的输出
middle_sequences = []
# 计算第一层的所有时间戳上的输出，并保存
for xt in tf.unstack(x, axis=1):
     out0, h0 = cell0(xt, h0)
     middle_sequences.append(out0)
# 计算第二层的所有时间戳上的输出
# 如果不是末层，需要保存所有时间戳上面的输出
for xt in middle_sequences:
    out1, h1 = cell1(xt, h1)

使用这种方式的话，我们需要一个额外的 List 来保存上一层所有时间戳上面的状态信息：
middle_sequences.append(out0)。这两种方式效果相同，可以根据个人喜好选择编程风格。